## Create FLUT

In [1]:
import sys
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
#import collections
import pandas as pd
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

# import netCDF4
# from netCDF4 import *

import cartopy.crs as ccrs
import cartopy as cart
import cartopy.mpl.ticker as cticker
import cartopy.feature as cfeature
from scipy import interpolate
from scipy.interpolate import griddata
import time
import glob
import dask

from scipy.fftpack import fft
from scipy.fftpack import ifft
import copy
import eofs.standard as Eof_st
from matplotlib.colors import ListedColormap

from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import cmocean

In [2]:
if 'client' in locals():
    client.shutdown()
    print('...shutdown client...')
else:
    print('client does not exist yet')

client does not exist yet


In [3]:
from distributed import Client
from ncar_jobqueue import NCARCluster

cluster = NCARCluster(project='NAML0001',walltime='12:00:00')
cluster.scale(40)
client = Client(cluster)
client

## Useful Functions

In [3]:
#this functions pick data ranges out of an xarray data structure.
def is_dayofyear(dayofyear,dd):
    return (dayofyear==dd)

def is_dayofyear_range(dayofyear,dd1,dd2):
    
    if (dd2 > dd1) & (dd2<366) & (dd1>=1):
        return ((dayofyear>=dd1)&(dayofyear<=dd2))
    elif (dd2 > 366) & (dd1>0):
        dd2 = dd2 - 366
        print('you are wrapping around years... this is a warning if you arent intending to')
        return ((dayofyear>=dd1)|(dayofyear<=dd2))
    elif dd1 < 1:
        print('you are wrapping around years... this is a warning if you arent intending to')
        dd1 = dd1+366
        return ((dayofyear>=dd1)|(dayofyear<=dd2))
    else:
        print('this is an edge case you did not foresee')
        return np.nan
    
def cal_ano_dcli(var):
    '''return [var_ano, var_dcli], Taxis= axis of time'''
    var_dcli=var.groupby('time.dayofyear').mean(dim='time')
    var_ano= var - var.mean(dim = 'time')
    
    return var_ano, var_dcli




#make MJO base line months
def is_NDJFM(dayofyear):
    return ((dayofyear==11)|(dayofyear==12)|(dayofyear==1)|(dayofyear==2)|(dayofyear==3))



def is_doyrange(doy,dd,tod,hh):
    daywind = 15
    if (dd - daywind) < 1:
        # print('in 1')
        return ((doy >= (366+(dd-daywind))) | (doy <= dd+daywind)) & (tod==hh)
        
    elif (dd + daywind) > 366:
        # print('in 2')
        return ((((doy <= 366) & (doy>=(dd-daywind))) | (doy <= (-1*((366-(dd-daywind))-(2*daywind)))  )))&(tod==hh)
    
    else:
        # print('in 3')
        return ((doy >= dd-daywind) & (doy <= dd+daywind))&(tod==hh)
    

# def is_NDJFM(dayofyear):
#     return ((dayofyear==12)|(dayofyear==1)|(dayofyear==2))

# def is_NDJFM(dayofyear):
#     return ((dayofyear==1)|(dayofyear==2)|(dayofyear==3)|(dayofyear==4)|(dayofyear==5)|(dayofyear==6)|(dayofyear==7)|(dayofyear==8)|(dayofyear==9)|(dayofyear==10)|(dayofyear==11)|(dayofyear==12))

#  NDJFM_data = GPH_E20c.sel(time=is_NDJFM(GPH_E20c['time.month']))

## First Load the CESM LENS Data
need Z500


## w/o dask

In [3]:
DSflut =  xr.open_dataset('/glade/scratch/wchapman/data_for_KJM/FLUT_CESM2_100_400.nc')
print('...loading...')
# DSflut.load()

...loading...


In [25]:
DSflut =  xr.open_dataset('/glade/scratch/wchapman/data_for_KJM/FLUT_CESM2_100_400.nc')
DSflut

<xarray.Dataset>
Dimensions:       (lat: 192, lon: 288, time: 109500, lev: 32, ilev: 33, nbnd: 2)
Coordinates:
  * lat           (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon           (lon) float64 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * lev           (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * ilev          (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * time          (time) object 0101-01-01 00:00:00 ... 0400-12-31 00:00:00
Dimensions without coordinates: nbnd
Data variables: (12/26)
    gw            (time, lat) float64 ...
    hyam          (time, lev) float64 ...
    hybm          (time, lev) float64 ...
    P0            (time) float64 ...
    hyai          (time, ilev) float64 ...
    hybi          (time, ilev) float64 ...
    ...            ...
    n2ovmr        (time) float64 ...
    f11vmr        (time) float64 ...
    f12vmr        (time) float64 ...
    sol_tsi       (time) float64 ...
    nsteph        (time) float64 ...
    FLUT          (time, lat, lon) float32 ...
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              b.e21.B1850.f09_g17.CMIP6-esm-piControl.001
    logname:           cmip6
    host:              cheyenne5
    initial_file:      b.e21.B1850_BPRP.f09_g17.CMIP6-piControl.tst.004.cam.i...
    topography_file:   /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/fv_0.9x1...
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    time_period_freq:  day_1

In [22]:
%%time
def cal_ano_dcli(var):
    '''return [var_ano, var_dcli], Taxis= axis of time'''
    var_dcli=var.groupby('time.dayofyear').mean(dim='time')
    var_ano= var.groupby('time.dayofyear') - var_dcli    
    return xvar_ano, var_dcli

DSz500_anom,DSz500_clim = cal_ano_dcli(DSz500)
print('....adding states...')
DSz500['Z500_anom']=DSz500_anom['Z500']
DSz500['Z500_clim'] = DSz500['Z500']-DSz500['Z500_anom']
DSz500.to_netcdf('/glade/scratch/wchapman/data_for_KJM/'+'Z500_CESM2_BHIST_100_400_KJMmethod.nc')

## Make Climatology
- This takes about 1000 years.. it's better to just load the files if they have already been made

In [1]:
%%time
DSflut =  xr.open_dataset('/glade/scratch/wchapman/data_for_KJM/FLUT_CESM2_100_400.nc')
print('...loading...')

def is_doyrange(doy,dd,tod,hh):
    daywind = 15
    if (dd - daywind) < 1:
        # print('in 1')
        return ((doy >= (366+(dd-daywind))) | (doy <= dd+daywind)) & (tod==hh)
        
    elif (dd + daywind) > 366:
        # print('in 2')
        return ((((doy <= 366) & (doy>=(dd-daywind))) | (doy <= (-1*((366-(dd-daywind))-(2*daywind)))  )))&(tod==hh)
    
    else:
        # print('in 3')
        return ((doy >= dd-daywind) & (doy <= dd+daywind))&(tod==hh)

print('copying')
flut_climo = xr.zeros_like(DSflut['FLUT'])
print('copied')
print('... creating climo from centered 30 day average ...')
for ee,dayhr in (enumerate(DSflut.time)):
    if ee%1 ==0:
        print('doing ',ee,' of 365')
    dooDOY = dayhr['time.dayofyear']
    hh=dayhr['time.hour']
    Dtemp = DSflut.sel(time=is_doyrange(DSflut['time.dayofyear'],dooDOY,DSflut['time.hour'],hh))[['FLUT']].mean(['time'])
    flut_climo[ee,:,:] = Dtemp['FLUT'].values
    
    if ee == (365)+8:
        endee=ee
        enddate = dayhr
        break

print("this is much faster but only works with noleap calendar:")  
flutnp = np.array(flut_climo[:365,:,:])
arrays_to_concat = [flutnp] * 300
concatenated_arr = np.concatenate(arrays_to_concat, axis=0)
flut_climo[:,:,:]=concatenated_arr
dr = pd.date_range(start='1801-01-01', end='2100-12-31', freq='D')
dates = dr[(dr.day != 29) | (dr.month != 2)]
DSflut['FLUT_climo'] = flut_climo
DSflut['time']=dates
svname = '/glade/scratch/wchapman/data_for_KJM//'+'FLUT_CESM2_100_400_repeats.nc'
DSflut.to_netcdf(svname)
print('otherwise you have to do this: (uncomment below)')

# print('...now repeating climo...') 
# for ee,dayhr in (enumerate(DSz500.time)):
#     dm = dayhr['time.month'].values
#     dd = dayhr['time.day'].values
#     dy = DSz500['time.year'][0].values

#     getday = f'{dy:04}'+'-'+f'{dm:02}'+'-'+f'{dd:02}'
    
#     #leap year shenanigans:
#     if f'{dm:02}'+'-'+f'{dd:02}' == '01-01': 
#         getday = f'{dy:04}'+'-01-02'
        
#     if ee%500 ==0:
#         print('doing ',ee,' of ', len(DSz500['time']))
#     try:
#         DtempZ500 = Z500_climo.sel(time=getday).squeeze()
#     except: 
#         DtempZ500 = Z500_climo.sel(time=getday,method='nearest').squeeze()
#     Z500_climo[ee,:,:] = DtempZ500.values
    
# DSz500['Z500_climo_v2'] = Z500_climo

# svname = '/glade/scratch/wchapman/data_for_KJM/'+'Z500_CESM2_BHIST_100_400_WECmethod.nc'
# DSz500.to_netcdf(svname)

## For Tony to do.. Create your anomaly: 
- subtract Z500_climo from Z500 and save a new variable Z500a
- Take and average over the north Pacific (lat[20,70],lon[150-240])
- - This will make the "index" of interest